# Interface

See the article [implementing an interface in python](https://realpython.com/python-interface/) on realpython.org.

An interface is some abstraction that allows you to define what methods must necessarily be implemented in successors.

In [15]:
class PlusConcatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        return f"{self.left}+{self.right}"

class MinusConcatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        return f"{self.left}-{self.right}"

print(
    "Plus concatenation",
    PlusConcatenation("hello", "world").get_output()
)
print(
    "Minus concatenation",
    MinusConcatenation("hello", "world").get_output()
)

Plus concatenation hello+world
Minus concatenation hello-world


In [20]:
class MetaConcatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        pass

class PlusConcatenation(MetaConcatenation):
    def get_output(self):
        return f"{self.left}+{self.right}"
class MinusConcatenation(MetaConcatenation):
    def get_output(self):
        return f"{self.left}-{self.right}"
print(
    "Plus concatenation",
    PlusConcatenation("hello", "world").get_output()
)
print(
    "Minus concatenation",
    MinusConcatenation("hello", "world").get_output()
)

Plus concatenation hello+world
Minus concatenation hello-world


## Using metaclasses

Идея создания интерфейса через метакласс, сосотои в том, что интерфейс имеет метакласс, в котором переопределены. `__instancecheck__` и `__subclasscheck__` (подробнее об этих базовых методах можно узать тут https://github.com/Dranikf/knowledge_bank/blob/main/python_class_spesials/python_class_specials.ipynb в разделе "Методы->Методы метаклассов"). Приведенным ниже образом.

In [22]:
class ParserMeta(type):
    '''
        Мета-парсер который будет использоваться для
        создания парсеров
    '''
    def __subclasscheck__(cls, subclass):
        '''
        Все классы наследующие этот класс 
        в качесве мета класса будут, будут своими 
        экземплярами (в смысле функции issubclass) 
        воспринимать лишь те классы, в которых объявлены 
        и определены методы load_data_source и extract_text.
        '''
        return (
            hasattr(subclass, 'load_data_source') and
            callable(subclass.load_data_source) and
            hasattr(subclass, 'extract_text') and
            callable(subclass.extract_text)
        )
    
    
    def __instancecheck__(cls, instance):
        '''
            Все классы наследующие это класс в качестве
            мета класса, своими экземплярами будут воспринимпть
            лишь те объекты, классы которых воспинимаются
            наследниками
        '''
        return cls.__subclasscheck__(type(instance))
    

    
class UpdatedInformalParserInterface(metaclass = ParserMeta):
    '''
    Объявляем обновленный парсер-интерфейс
    '''
    pass

Рассмотрим, тот-же пример.

In [27]:
class PdfParserNew():
    '''
        Новая конкредная реализация парсера для pdf документов
    '''
    def load_data_source(self, path: str, file_name:str) -> str:
        '''
            Определяем как именно должен работать парсинг
            для pdf документов
        '''
        pass
    
    def extract_text(self, full_file_path:str) -> dict:
        '''
            Тут также определяем как именно должен работать 
            парсинг для pdf документов
        '''
        pass
    
class EmlParserNew:
    '''
        Конкретная реализация для документов электронной почты
    '''
    def load_data_source(self, path:str, file_name:str)->str:
        '''
            Определяем как именно должен работать парсинг для email документов
        '''
        pass
    
    def extract_text_from_email(self, full_file_path:str)->str:
        '''
            Метод определенный только для документов email,
            но он по прежнему определяет как должен работать
            парсинг для email документов
        '''
        pass

Проверяем результат выполнения функции `issubclass` для новосозданных классов.

In [24]:
issubclass(PdfParserNew, UpdatedInformalParserInterface)

True

In [25]:
issubclass(EmlParserNew, UpdatedInformalParserInterface)

False

Так формально `UpdatedInformalParserInterface` не является реализацией интерфейса `EmlParserNew`.

**Но такая реализация по прежнему не являтся правильной**<br>
Рассмотрим результат метода `__mro__` для `PdfParserNew` (`__mro__` - одно из <a href="https://github.com/Dranikf/knowledge_bank/blob/main/python_class_spesials/python_class_specials.ipynb">специальных полей</a>).

In [26]:
PdfParserNew.__mro__

(__main__.PdfParserNew, __main__.UpdatedInformalParserInterface, object)

Так в `__mro__` для класса `PdfParserNew`, не видно, что он как-либо связан с `UpdatedInformalParserInterface`. Такую ситуалию еще описывают, что `UpdatedInformalParserInterface` является виртуальным базовым классом для класса `PdfParserNew`.

Впрочем, новерное, это можно преодолеть следующей реализацией pdf парсера.

In [30]:
class PdfParserNew2(UpdatedInformalParserInterface):
    '''
        Новая конкредная реализация парсера для pdf документов
    '''
    def load_data_source(self, path: str, file_name:str) -> str:
        '''
            Определяем как именно должен работать парсинг
            для pdf документов
        '''
        pass
    
    def extract_text(self, full_file_path:str) -> dict:
        '''
            Тут также определяем как именно должен работать 
            парсинг для pdf документов
        '''
        pass

In [31]:
issubclass(PdfParserNew2, UpdatedInformalParserInterface)

True

In [30]:
PdfParserNew2.__mro__

(__main__.PdfParserNew2, __main__.UpdatedInformalParserInterface, object)

# Формальная реализация интерфейса

Для формальной реализации интерфейсов используется модуль `abc`

In [ ]:
import abc

